In [106]:
import numpy as np
import json_tricks


inputs = json_tricks.load('inputs/inputs.json')
answer = {}

# Contra- and Co- Variant Coordinates (Task)

Find convariant and contravariant coordinates of the vectors:

$$\mathbf x = \begin{bmatrix}
1 \\ 2 \\ 3
\end{bmatrix}, 

\mathbf y = \begin{bmatrix}
3 \\ 4 \\ 5
\end{bmatrix}$$

in two sets of basis:

$$
\begin{bmatrix}
\mathbf a_1 & \mathbf a_2 & \mathbf a_3
\end{bmatrix}
=
\begin{bmatrix}
1 & 1 & 1 \\
0 & 1 & 1 \\
0 & 0 & 1 \\
\end{bmatrix}
$$

$$
\begin{bmatrix}
\mathbf b_1 & \mathbf b_2 & \mathbf b_3
\end{bmatrix}
=
\begin{bmatrix}
1 & 0 & 0 \\
1 & 1 & 0 \\
1 & 1 & 1 \\
\end{bmatrix}
$$

In [107]:
answer['task0'] = {
    'a': {
        'contravariant_x': np.array([
            -1, -1, 3
        ]),
        'contravariant_y': np.array([
            -1, -1, 5
        ]),
        'covariant_x': np.array([
            1, 3, 6
        ]),
        'covariant_y': np.array([
            3, 7, 12
        ])
    },
    'b': {
        'contravariant_x': np.array([
            1, 1, 1
        ]),
        'contravariant_y': np.array([
            3, 1, 1
        ]),
        'covariant_x': np.array([
            6, 5, 3
        ]),
        'covariant_y': np.array([
            12, 9, 5
        ])
    }
}

# 3-dimensional arrays

It is time to start working with 3 dimensional arrays. That
might be useful when you want to speed up some calculations.

## Task

You are given a set of $N$ basis vectors in the form of row-matrix $B$ of size $N \times M$ and a set of **contravariant** coordinates in that basis $C$ for $K$ vectors of size $N \times K$:

$$B = 
\begin{bmatrix}
- & \mathbf b_1 & - \\
- & \mathbf b_2 & - \\
  & \dots & \\
- & \mathbf b_N & -
\end{bmatrix}$$

$$C =
\begin{bmatrix}
c_1^1 & c_1^2 & \dots & c_1^K \\
c_2^1 & c_2^2 & \dots & c_2^K \\
\vdots & \vdots & \ddots & \vdots \\
c_N^1 & c_N^2 & \dots & c_N^K \\
\end{bmatrix}
$$

Thus, $N$ is the number of the basis
vectors and, thus number of coordinates in the span,
$M$ is the dimensionality of the space, $K$ is the number of vectors to reconstruct.

**The task is** to reconstruct the vectors that are defined using these coordinates. The answer should be
a matrix of shape $K \times M$

**You should not use the cycles in this task**

# Interface

## Inputs

`B, C` -- 2D `numpy` arrays

## Outputs

`V` -- 2D `numpy` array


<details>
<summary> <b>Hint</b> </summary>
So, how are you supposed to deal with this task?

If you are allowed to use cycles, that is rather simple: you
iterate over the basis vectors and corresponding coordinates,
multiply one with another and collect the result one-by-one.

But the trick here is that you should do that in a vectorized
manner. So, how to deal with it?

We want in the end to recieve an array of the size 
$K \times M$.

And here is how this get it:
1) reshape your basis vectors to the size of 
$N \times 1 \times M$. It is non-trivial to do that with
usual `reshape`, so there is a function 
`numpy.expand_dims(x, position)` that allows to add a
dimension of an array to the specified position. In this
case the `position=1` as we want to insert dimension of
size `1` to the first position.
2) reshape the coordinates in this basis to the size of
$N \times K \times 1$. Use `numpy.expand_dims` yet again.
3) multiply two tensors from above and receive array of size
$N \times K \times M$. This will first cast the shapes of 
the abovementioned arrays and then perform the multiplication.
This result holds all the required products you may ever need.
4) Now sum this array along the `axis=0` to get $K \times M$
array.
5) Enjoy

Note that although this approach works really fast, it
requires quite a lot of memory. There is another way of doing
this operation that we will study later.
</details>

In [108]:
def reconstruct_vectors(B, C):
    V = B.T @ C
    return V


B: each row is a basis vector (shape: N × M)


C: each column is a set of coordinates (shape: N × K)


When doing B.T @ C, the result V has shape (M, K), so each column of V is a rebuilt vector in the standard basis.

The function rebuilds vectors using special basis vectors (from B) and their coordinates (from C).


Each row of B is a basis vector.


Each column of C tells how much of each basis vector to use.


The function uses matrix multiplication to combine them quickly.


The result is a new matrix where each column is one of the rebuilt vectors in the standard basis.

In [109]:
answer['task1'] = []
for input in inputs['task1']:
    res = reconstruct_vectors(**input)
    answer['task1'].append(res)

# Task

You are given a set of vectors as basis that are stored
in a row-matrix `numpy.ndarray` called `B` and a set of vectors
`X`. It is guaranteed that the vectors from `X` come from
the span of the `B` vectors. Find **covariant** coordinates
of vectors `X` in this `basis`.

## Interface

**Inputs**: `basis: numpy.ndarray`

**Outputs**: `coords: numpy.ndarray`

In [110]:
def get_covariant_coordinates(B, X):
    # B: (k, n), X: (m, n)
    # We solve for C: (m, k) such that C @ B = X
    B_pinv = np.linalg.pinv(B)
    coords = X @ B_pinv.T
    return coords

In [111]:
answer['task2'] = []
for input in inputs['task2']:
    answer['task2'].append(res)

# Task

For the basis vectors and contravariant coordinates from task #1, find covariant coordinates

In [112]:
def contravariant_to_covariant(B, C):
    B_pinv = np.linalg.pinv(B)  # shape: (n, k)

    # compute coordinates
    coords = X @ B_pinv        # shape: (m, k)
    return coords

In [113]:
answer['task3'] = []
for input in inputs['task1']:
    #res = contravariant_to_covariant(**input)
    answer['task3'].append(res)

In [114]:
json_tricks.dump(answer, '.answer.json')

'{"task0": {"a": {"contravariant_x": {"__ndarray__": [-1, -1, 3], "dtype": "int64", "shape": [3]}, "contravariant_y": {"__ndarray__": [-1, -1, 5], "dtype": "int64", "shape": [3]}, "covariant_x": {"__ndarray__": [1, 3, 6], "dtype": "int64", "shape": [3]}, "covariant_y": {"__ndarray__": [3, 7, 12], "dtype": "int64", "shape": [3]}}, "b": {"contravariant_x": {"__ndarray__": [1, 1, 1], "dtype": "int64", "shape": [3]}, "contravariant_y": {"__ndarray__": [3, 1, 1], "dtype": "int64", "shape": [3]}, "covariant_x": {"__ndarray__": [6, 5, 3], "dtype": "int64", "shape": [3]}, "covariant_y": {"__ndarray__": [12, 9, 5], "dtype": "int64", "shape": [3]}}}, "task1": [{"__ndarray__": [[-155, -85, -35, -36, -65, 60], [96, -43, -123, -110, 5, 72], [27, -37, -89, 114, 36, -93], [19, 60, 15, -26, 72, 132], [124, -89, -146, -14, -186, 56], [111, 72, 30, -87, 49, 280], [-59, -112, -38, -20, -111, -16], [-59, 170, 141, -9, 57, 7], [163, 146, 119, 65, 14, 25]], "dtype": "int64", "shape": [9, 6], "Corder": true}